In [17]:
import time
i = 0

start_time = time.time()
for _ in range(10):
    time.sleep(0.1)
    print(f"{-start_time+(start_time := time.time())} seconds have passed")
    # print(f"{(start_time := time.time())-start_time} seconds have passed")


0.10034847259521484 seconds have passed
0.10036730766296387 seconds have passed
0.10026788711547852 seconds have passed
0.10023331642150879 seconds have passed
0.10031914710998535 seconds have passed
0.10023617744445801 seconds have passed
0.10024189949035645 seconds have passed
0.10032510757446289 seconds have passed
0.10022377967834473 seconds have passed
0.10019159317016602 seconds have passed


In [1]:
class Foo():
    def __init__(self):
        self.__bar = 42

foo = Foo()
foo.__dict__

{'_Foo__bar': 42}

In [1]:
import bittensor as bt

In [3]:
metagraph = bt.metagraph(netuid=61, network="test", sync=True, lite=False)

In [8]:
metagraph.axons

[AxonInfo( /ipv4/129.146.127.82:8091, 5DhKVLE5bsg8BYY7QC94AJzGHC1LJQ9oCW4R33GekuVMqDY1, 5G1XChmwskSD26xgmdPsJxd4t9EP1CCTHA5KvSNN6uJ73N8D, 660 ),
 AxonInfo( /ipv4/138.84.62.175:8091, 5CdJRFCwnahgTKquJ92HnbCdp2tb73LmPamvMtWfEUH1cdkp, 5G1XChmwskSD26xgmdPsJxd4t9EP1CCTHA5KvSNN6uJ73N8D, 681 ),
 AxonInfo( /ipv4/129.146.127.82:9002, 5H9AVNRo135552roWVL5BW8rLcoBi9HrRSQnrgfipE8vXdSH, 5G1XChmwskSD26xgmdPsJxd4t9EP1CCTHA5KvSNN6uJ73N8D, 660 ),
 AxonInfo( /ipv4/129.146.127.82:9003, 5Gk23dLJVdTpLU1fi1oAnkghc3VJQuPe2dQEJNUR2VqnBLZv, 5G1XChmwskSD26xgmdPsJxd4t9EP1CCTHA5KvSNN6uJ73N8D, 660 ),
 AxonInfo( /ipv4/129.146.127.82:9004, 5F1dDnWxQGnBCE5VDa7RvnPnKg7SYGdvEoLuf2mjnBu43WDJ, 5G1XChmwskSD26xgmdPsJxd4t9EP1CCTHA5KvSNN6uJ73N8D, 660 ),
 AxonInfo( /ipv4/129.146.127.82:9005, 5CdT2QvbpsWEiCu7ZkZr7XZG27vkQUg4mqdpTEeHSGXoZD6p, 5G1XChmwskSD26xgmdPsJxd4t9EP1CCTHA5KvSNN6uJ73N8D, 660 ),
 AxonInfo( /ipv4/129.146.127.82:9006, 5DrEZkh5KrpC7GpyXKy8ZKSj35iKHetniDnYg8Q6NJuTynAE, 5G1XChmwskSD26xgmdPsJxd4t9EP1CCTHA5KvSNN6uJ

In [18]:
import numpy as np


x = np.reshape(np.random.random(size=10), (1,-1))
y = np.reshape(np.random.random(size=10), (-1,1))

float(x@y), x@y

/tmp/ipykernel_95078/659430835.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  float(x@y), x@y


(2.561555106557572, array([[2.56155511]]))

In [5]:
# from prompting.tasks.date_qa import DateQuestionAnsweringTask
from prompting.tasks.base_task import BaseTask
from prompting.rewards.reward import BaseRewardModel
from prompting.tasks.summarization import SummarizationTask, SummarizationRewardConfig
from prompting.tasks.qa import QuestionAnsweringTask, QARewardConfig

from prompting.datasets.wiki import WikiDataset
from prompting.datasets.base import BaseDataset
from pydantic import BaseModel, ConfigDict
import random
from typing import ClassVar
import bittensor as bt


class TaskConfig(BaseModel):
    task: BaseTask.__class__
    probability: float
    datasets: list[BaseDataset.__class__]
    reward_model: BaseRewardModel.__class__

    model_config = ConfigDict(arbitrary_types_allowed=True)


class TaskRegistry(BaseModel):
    tasks: ClassVar[list[TaskConfig]] = [
        TaskConfig(task=QuestionAnsweringTask, probability=0.6, datasets=[WikiDataset], reward_model=QARewardConfig),
        TaskConfig(
            task=SummarizationTask, probability=0.4, datasets=[WikiDataset], reward_model=SummarizationRewardConfig
        ),
        # TaskConfig(task=DateQuestionAnsweringTask, probability=0.2, datasets=[WikiDateDataset])
    ]

    @classmethod
    def random(cls) -> TaskConfig:
        probabilities = [task.probability for task in cls.tasks]
        selected_task = random.choices(cls.tasks, probabilities)[0]
        return selected_task

    @classmethod
    def get_task_datasets(cls, task: BaseTask.__class__):
        try:
            return [t.datasets for t in cls.tasks if task is t.task][0]
        except Exception:
            bt.logging.error("Tried accessing non-registered task")
            return []

    @classmethod
    def get_random_task_dataset(cls, task: BaseTask.__class__) -> BaseDataset.__class__:
        return random.choice(cls.get_task_datasets(task))

    @classmethod
    def get_task_reward(cls, task: BaseTask) -> BaseRewardModel:
        try:
            return [t.reward_model for t in cls.tasks if task is t.task][0]
        except Exception:
            bt.logging.error("Tried accessing non-registered task")
            return []

    @classmethod
    def create_random_task(cls, llm_pipeline) -> BaseTask:
        task_config = cls.random()
        dataset = cls.get_random_task_dataset(task_config.task)
        return task_config.task(
            llm_pipeline=llm_pipeline, context=dataset().next(), reward_config=task_config.reward_model()
        )


/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-05 10:47:00,850	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "name" in "WikiDateDataset" shadows an attribute in parent "BaseDataset"
  warnings.warn(


In [9]:
TaskRegistry().get_task_reward(QuestionAnsweringTask)

ValidationError: 1 validation error for QuestionAnsweringTask
context
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing